In [63]:
import sys
sys.path.append('../../')

import pandas as pd
from utils import fill_empty, VColumns
from dotenv import load_dotenv
load_dotenv()

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# First file

In [64]:
df1 = pd.read_csv("v0-preprocessed.csv")

df1.head()

,external_id,name,name_2,name_3,internal_notes,vintage,info,bottle_qty,magnum_qty,demi_qty,bottle_price,magnum_price,demi_price,Unnamed: 13,Unnamed: 14
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Passito,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Passito,Cashmere Gewürztraminer,Weingut Elena Walch,PASSITO,2020,NaN,0.0,NaN,NaN,19.00 €,NaN,NaN,NaN,NaN


In [65]:
# Create a new dataframe to store the expanded rows
expanded_rows = []

# Iterate through each row in the original dataframe
for _, row in df1.iterrows():
    # Add the original row
    base_row = {
        'external_id': row['external_id'],
        'name': ' '.join(filter(None, [
            str(row['name']) if pd.notna(row['name']) else '',
            str(row['name_2']) if pd.notna(row['name_2']) else '',
            str(row['name_3']) if pd.notna(row['name_3']) else ''
        ])).strip(),
        'vintage': row['vintage'],
        'info': row['info'],
        'internal_notes': row['internal_notes']
    }
    
    # Check each bottle size and add new rows if quantity exists
    bottle_qty = 0 if pd.isna(row['bottle_qty']) else float(row['bottle_qty'])
    if bottle_qty > 0:
        bottle_row = base_row.copy()
        bottle_row['size'] = 'BOTTLE'
        bottle_row['price'] = row['bottle_price']
        bottle_row['quantity'] = bottle_qty
        expanded_rows.append(bottle_row)
        
    magnum_qty = 0 if pd.isna(row['magnum_qty']) else float(row['magnum_qty'])
    if magnum_qty > 0:
        magnum_row = base_row.copy()
        magnum_row['size'] = 'MAGNUM'
        magnum_row['price'] = row['magnum_price']
        magnum_row['quantity'] = magnum_qty
        expanded_rows.append(magnum_row)
        
    demi_qty = 0 if pd.isna(row['demi_qty']) else float(row['demi_qty'])
    if demi_qty > 0:
        demi_row = base_row.copy()
        demi_row['size'] = 'HALF_BOTTLE'
        demi_row['price'] = row['demi_price']
        demi_row['quantity'] = demi_qty
        expanded_rows.append(demi_row)

# Create the new dataframe with expanded rows
df = pd.DataFrame(expanded_rows)

df.head()

,external_id,name,vintage,info,internal_notes,size,price,quantity
0,NaN,Passito Ornus dell Ornelaia Petit Manseng,2016,NaN,PASSITO,BOTTLE,19.00 €,26.0
1,NaN,Ornus dell Ornelaia Petit Manseng,2015,NaN,NaN,BOTTLE,NaN,4.0
2,NaN,Ornus dell Ornelaia Petit Manseng,2011,NaN,NaN,BOTTLE,NaN,6.0
3,NaN,Ornus dell Ornelaia Petit Manseng,2010,NaN,NaN,BOTTLE,NaN,3.0
4,NaN,Ornus dell Ornelaia Petit Manseng,2009,NaN,NaN,BOTTLE,NaN,1.0


In [66]:
# df to csv
df.to_csv("v0_postprocessed.csv", index=False)